# **TP Intelligence Artificielle - SAT et CSP**

# **Partie 0 : Visualisation**

Voici une fonction pour visualiser les sudokus.

In [2]:
from IPython.display import HTML

def visualize_sudoku(grid):
    html = "<table>"
    for i in range(9):
        html += "<tr>"
        for j in range(9):
            value = grid[i][j]
            cell_style = "width: 30px; height: 30px; text-align: center; font-size: 20px; border: 1px solid black;"
            if i % 3 == 0 and i != 0:  # Horizontal lines
                cell_style += "border-top: 5px solid black;"
            if j % 3 == 0 and j != 0:  # Vertical lines
                cell_style += "border-left: 5px solid black;"

            if value == 0:
                html += f"<td style='background-color: white; {cell_style}'> </td>"  # Case vide
            else:
                html += f"<td style='background-color: rgb(226, 114, 91); {cell_style}'>{value}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### **Exercice 1**
Écrivez un Sudoku sous forme d’une liste de listes (9 listes de 9 élements chacune où la valeur 0 represente une caise vide), puis utilisez la fonction visualize_sudoku pour l’afficher.

In [3]:
# 创建一个数独棋盘
grid = [  [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

print('Sudoku')
visualize_sudoku(grid)

Sudoku


5,3,,,7,,,,
6,,,1,9,5,,,
,9,8,,,,,6,
8,,,,6,,,,3
4,,,8,,3,,,1
7,,,,2,,,,6
,6,,,,,2,8,
,,,4,1,9,,,5
,,,,8,,,7,9


# **Partie 1 : Modélisation**

Nous allons modéliser le Sudoku comme un problème SAT et CSP, puis utiliser les méthodes propres à chaque modélisation pour la résolution.
Pour cela, nous utiliserons deux bibliothèques : **python-sat** et **python-constraint**.

Exécutez le bloc de code suivant pour les installer.
Pour les installer sur vos PC, il suffit d’exécuter les mêmes lignes (sans le '!' au début) dans une console de commandes (CMD).



In [5]:
# 把数独模型建为一个 SAT 问题 和 一个 CSP 问题
# Installation des libraries 导入库
!pip install python-sat # 用于构造和解决 SAT 问题
!pip install python-constraint # 用于构造和解决 CSP 问题

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.5 MB/s eta 0:00:00


## **1.1 Modélisation SAT**

Pour modéliser le Sudoku comme un problème SAT, nous utiliserons la méthode CNF de la bibliothèque pysat.formula.
CNF permet de définir les clauses logiques représentant les contraintes du problème.
Pour les variables, nous utiliserons un dictionnaire.

In [8]:
# 从 pysat库中导入CNF类，用来表示逻辑公式的合取范式 Conjunctive Normal Form
from pysat.formula import CNF

class Sudoku_SAT:

    def __init__(self, grid):
        self.grid = grid
        self.cnf = CNF()  # 创建一个 CNF 对象，用于存储所有逻辑子句
        self.var_map = {}  # 用于将 (行, 列, 数字) 映射到 SAT 变量编号

        # 1.创建SAT变量映射
        # 每个格子(i, j) 和每个数字 k 都被映射成一个 SAT 变量 ID（从1开始递增）
        var_count = 1
        for i in range(9): # 行
            for j in range(9): # 列
                for k in range(1, 10): # （1，10）在python中表示 1-9 #数值
                    self.var_map[(i, j, k)] = var_count
                    var_count += 1

        # 2. Créer les clauses SAT
        # a. 每个格子至少填一个数字
        for i in range(9):
            for j in range(9):
                clause = [self.var_map[(i, j, k)] for k in range(1, 10)]
                self.cnf.append(clause)
                #向表格中填写数字 -- 向 CNF 公式中添加一个子句（clause）

        # b. Chaque case ne peut contenir qu'une seule valeur
         # 保证每个格子只有一个值： 逻辑子句：¬x_ijk1 OR ¬x_ijk2)
        for i in range(9):
            for j in range(9):
                for k1 in range(1, 10):
                    for k2 in range(k1 + 1, 10):
                        clause = [-self.var_map[(i, j, k1)], -self.var_map[(i, j, k2)]]
                        self.cnf.append(clause)

        # c. Chaque ligne doit contenir tous les chiffres de 1 à 9
        for i in range(9):
          # 每行必须包含每个数字
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for j in range(9)]
                self.cnf.append(clause)

        # d. Chaque colonne doit contenir tous les chiffres de 1 à 9
        for j in range(9):
          #每列必须包含每个数字
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for i in range(9)]
                self.cnf.append(clause)

        # e. Chaque sous-grille 3x3 doit contenir tous les chiffres de 1 à 9
        for subgrid_row in range(3):
            for subgrid_col in range(3):
              # 个 3x3 小方块必须包含每个数字
                for k in range(1, 10):
                    clause = [self.var_map[(subgrid_row * 3 + i, subgrid_col * 3 + j, k)]
                              for i in range(3) for j in range(3)]
                    self.cnf.append(clause)

        # f. Contraintes initiales de la grille
        # 添加初始数独中的已知数字
        # 如果某个格子已经填了数字，比如 grid[i][j] = 5，那么直接添加子句 (x_ij5)
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] != 0:
                    self.cnf.append([self.var_map[(i, j, self.grid[i][j])]])

### **Exercice 2**
Créez une fonction qui prend une grille de Sudoku en entrée et utilise le solveur Glucose3 de la bibliothèque pysat.solver pour résoudre le jeu (pensez à bien importer Glucosa3). Votre fonction doit :

1. Créer un objet sudoku_sat pour modéliser le problème.
2. Ajouter au solveur les contraintes du Sudoku sous forme de clauses logiques.
3. Vérifier si le Sudoku est résoluble en utilisant le solveur SAT.
4. Trouver une solution si le Sudoku est résoluble.
5. Interpréter la solution retournée par le solveur pour remplir la grille :
  * La solution sera une liste de taille 9 × 9 × 10, contenant des valeurs positives et négatives.
  * Une valeur positive signifie que la variable (i, j, k) est True, ce qui indique que la case (i, j) doit contenir la valeur k.

In [10]:
from pysat.solvers import Glucose3
# 这是 pysat 中性能很好的一个 SAT 解算器

def solve_sudoku_sat(grid):
  # 创建数独的 SAT 表示模型
  sudoku_sat = Sudoku_SAT(grid)

  # 创建 SAT 求解器 Glucose3
  solver = Glucose3()

  # 将 CNF 子句添加到求解器中
  solver.append_formula(sudoku_sat.cnf)

  # Solve the SAT problem
  is_satisfiable = solver.solve()

  # 如果有解，就获取解并还原为完整数独棋盘
  if is_satisfiable:
    solution = solver.get_model()
    # 如果数独有解，就调用 get_model() 获取一个解（也就是变量的取值）
    # 用于在调用 .solve() 之后获取一个可行解（model）

    grid = [[] for _ in range(9)]
    # 重新初始化 grid 为一个空的 9×9 列表，用来放求解结果。

    var_count = 0 # var_count 是变量编号索引
    for i in range(9):
        for j in range(9):
            for k in range(1, 10):
                if solution[var_count] > 0:
                  # solution[var_count] 是 SAT 求解器给出的第 var_count 个变量的值
                  # 大于0 -- 判定为true
                  grid[i].append(k)
                var_count += 1

    print('Solution SAT Sudoku')
    visualize_sudoku(grid)

### **Exercice 3**
Utilisez votre fonction pour resoudre le sudoku ci-dessous:

    [0, 0, 0, 0, 7, 0, 0, 0, 0]
    [6, 0, 0, 1, 9, 5, 0, 0, 0]
    [0, 9, 8, 0, 0, 0, 0, 6, 0]
    [8, 0, 0, 0, 6, 0, 0, 0, 3]
    [4, 0, 0, 8, 0, 3, 0, 0, 1]
    [7, 0, 0, 0, 2, 0, 0, 0, 6]
    [0, 6, 0, 0, 0, 0, 2, 8, 0]
    [0, 0, 0, 4, 1, 9, 0, 0, 5]
    [0, 0, 0, 0, 8, 0, 0, 7, 9]

In [12]:
grid = [[0, 0, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

print('Initial Sudoku')
visualize_sudoku(grid)

# Solve sudoku
solve_sudoku_sat(grid)

Initial Sudoku


,,,,7,,,,
6,,,1,9,5,,,
,9,8,,,,,6,
8,,,,6,,,,3
4,,,8,,3,,,1
7,,,,2,,,,6
,6,,,,,2,8,
,,,4,1,9,,,5
,,,,8,,,7,9


Solution SAT Sudoku


5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


## **1.2 Modélisation CSP**

### **Exercice 4**

Créez une fonction solve_sudoku_csp qui prend une grille de Sudoku en entrée, la modélise comme un problème CSP et la résout. Votre fonction doit :
1. Créer une instance du problème CSP
  * Utilisez la classe Problem de la bibliothèque python-constraint pour représenter le Sudoku.
2. Définir les variables du problème
  * Utilisez la méthode addVariable pour ajouter les variables au problème, sachant que :
    * Chaque case de la grille est une variable identifiée par un tuple (ligne, colonne).
    * Le domaine de chaque variable correspond aux valeurs possibles (de 1 à 9).
    * Si une case contient déjà un chiffre (non nul), son domaine est restreint à cette valeur unique.
3. Définir les contraintes du Sudoku (les valeurs doivent être uniques dans chaque ligne, colonne et région 3×3). Pour cela, utilisez la méthode addConstraint avec AllDifferentConstraint() pour assurer ces contraintes.
4. Résoudre le problème. Pour cela utilisez la méthode getSolutions() pour obtenir toutes les solutions possibles.
5. Gérer les solutions
  * Si des solutions existent, les stocker dans une liste.
  * Sinon, indiquer qu’aucune solution n’a été trouvée.

In [14]:
# 从 python-constraint 库中导入所有内容，包括 Problem, AllDifferentConstraint 等
from constraint import *

# 定义一个函数 solve_sudoku_csp，接收一个 9×9 数独棋盘 grid 作为输入
def solve_sudoku_csp(grid):
    problem = Problem()  # 创建一个 CSP 问题实例

    # Step 1: 定义变量（每个格子）以及它们的取值域（domain）
    for row in range(9):  # 遍历所有行
        for col in range(9):  # 遍历所有列
            if grid[row][col] == 0:
                # 如果当前格子为空（值为0），则取值域是1~9
                problem.addVariable((row, col), range(1, 10))
            else:
                # 如果当前格子已有值，取值域只包含这个值
                problem.addVariable((row, col), [grid[row][col]])

    # Step 2: 添加约束
    # 行约束：每一行的9个格子取值必须不同
    for i in range(9):
        problem.addConstraint(AllDifferentConstraint(), [(i, j) for j in range(9)])

    # 列约束：每一列的9个格子取值必须不同
    for i in range(9):
        problem.addConstraint(AllDifferentConstraint(), [(j, i) for j in range(9)])

    # 宫格约束：每个3×3的小宫格内的格子取值也必须不同
    for i in range(0, 9, 3):  # 每3行一组：i ∈ {0, 3, 6}
        for j in range(0, 9, 3):  # 每3列一组：j ∈ {0, 3, 6}
            block_cells = [(i + a, j + b) for a in range(3) for b in range(3)]
            problem.addConstraint(AllDifferentConstraint(), block_cells)

    # Step 3: 求解问题，获取所有可能的解
    solutions = problem.getSolutions()

    # Step 4: 判断是否有解
    if solutions:
        # 如果存在解，提取第一个解（字典形式），转换成 2D 数组形式
        first_solution = solutions[0]
        result_grid = [[0 for _ in range(9)] for _ in range(9)]
        for (row, col), value in first_solution.items():
            result_grid[row][col] = value
        return result_grid  # 返回填好数字的结果数独
    else:
        return None  # 如果无解，返回 None（你也可以改为抛出异常）


### **Exercice 5**
Utilisez votre fonction pour resoudre le jeu suivant.

In [15]:
# Grille de Sudoku (0 représente une case vide)
grid = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [17]:
# Créer une instance de Sudoku
print('Initial Sudoku')
visualize_sudoku(grid)

# Solving sudoku
solution = solve_sudoku_csp(grid)

print('Solution Sudoku')
visualize_sudoku(solution)

Initial Sudoku


5,3,,,7,,,,
6,,,1,9,5,,,
,9,8,,,,,6,
8,,,,6,,,,3
4,,,8,,3,,,1
7,,,,2,,,,6
,6,,,,,2,8,
,,,4,1,9,,,5
,,,,8,,,7,9


Solution Sudoku


5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


# **Partie 2 : Comparaison CSP vs SAT**

Nous allons comparer les deux solveurs sur 10 sudokus differents. Pour cela, voici 10 grilles.

In [22]:
grid1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [23]:
grid2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

In [24]:
grid3 = [
    [0, 2, 0, 6, 0, 8, 0, 0, 0],
    [5, 8, 0, 0, 0, 9, 7, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [3, 7, 0, 0, 0, 0, 5, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 8, 0, 0, 0, 0, 1, 3],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 9, 8, 0, 0, 0, 3, 6],
    [0, 0, 0, 3, 0, 6, 0, 9, 0]
]

In [25]:
grid4 = [
    [0, 0, 5, 3, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 7, 0, 0, 1, 0, 5, 0, 0],
    [4, 0, 0, 0, 0, 5, 3, 0, 0],
    [0, 1, 0, 0, 7, 0, 0, 0, 6],
    [0, 0, 3, 2, 0, 0, 0, 8, 0],
    [0, 6, 0, 5, 0, 0, 0, 0, 9],
    [0, 0, 4, 0, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 9, 7, 0, 0]
]

In [26]:
grid5 = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

In [27]:
grid6 = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

In [28]:
grid7 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

In [29]:
grid8 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9]
]

In [30]:
grid9 = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]

In [31]:
grid10 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

### **Exercice 6**

Créez une fonction comparer_solveurs qui prend comme entrée une liste de grilles, résout tous les jeux avec les deux solveurs, enregistre le temps d'exécution à chaque fois, et retourne les résultats sous forme de DataFrame.

Utilisez cette fonction pour résoudre tous les jeux et affichez ensuite les résultats.

In [33]:
# 导入时间模块和数据表处理模块
import time
import pandas as pd

# 定义函数 comparer_solveurs：比较多个数独用 CSP 和 SAT 方法的解题时间
def comparer_solveurs(grids):
    results = []  # 用于存放每个数独的运行结果

    for l, grid in enumerate(grids):  # 遍历 grids 中的每一个数独 grid，l 是编号
        print(f"正在求解第 {l + 1} 个数独...")

        # ========== 使用 CSP 方法 ==========
        print("CSP 解算器开始")
        start_time = time.time()  # 记录开始时间
        csp_solution = solve_sudoku_csp(grid)  # 调用你之前写的 CSP 求解函数
        csp_time = time.time() - start_time  # 计算运行时间（秒）

        # ========== 使用 SAT 方法 ==========
        print("SAT 解算器开始")
        start_time = time.time()  # 再次记录当前时间
        sudoku_sat = Sudoku_SAT(grid)  # 用 grid 创建一个 SAT 模型
        solver = Glucose3()  # 创建 SAT 解算器
        solver.append_formula(sudoku_sat.cnf)  # 把生成的 CNF 子句加进去
        is_satisfiable = solver.solve()  # 开始求解 SAT 问题

        if is_satisfiable:
            sat_solution = solver.get_model()  # 获取解（列表形式）
            sat_time = time.time() - start_time  # 计算运行时间

            # 还原 SAT 解为 9×9 的二维数独结果
            sat_solution_grid = [[0 for _ in range(9)] for _ in range(9)]
            var_count = 0  # 用来遍历解的变量编号
            for i in range(9):
                for j in range(9):
                    for k in range(1, 10):
                        if sat_solution[var_count] > 0:
                            sat_solution_grid[i][j] = k  # 如果变量为真，填入格子
                            break  # 找到一个值后跳出 k 循环
                        var_count += 1
        else:
            # 如果 SAT 无解，标记结果为 None，时间设为无穷大
            sat_solution_grid = None
            sat_time = float('inf')

        # 把当前数独的比较结果添加到结果列表中
        results.append([f"Sudoku {l + 1}", round(csp_time, 3), round(sat_time, 3)])

    # 把结果整理成一个 pandas DataFrame 表格，方便查看与分析
    df = pd.DataFrame(results, columns=["Sudoku", "CSP Time (s)", "SAT Time (s)"])
    return df  # 返回结果表格

# 假设你已经准备好了10个数独（grid1 到 grid10）
grid = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9, grid10]

# 调用函数进行求解和比较
df_results = comparer_solveurs(grid)

# 输出完成提示
print('所有数独已完成求解！')

正在求解第 1 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 2 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 3 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 4 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 5 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 6 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 7 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 8 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 9 个数独...
CSP 解算器开始
SAT 解算器开始
正在求解第 10 个数独...
CSP 解算器开始
SAT 解算器开始
所有数独已完成求解！


In [35]:
from tabulate import tabulate
#从 tabulate 库中导入 tabulate 函数。
#这个库用于将 列表 / 字典 / pandas 表格 格式化为漂亮的 ASCII 表格，可以用于终端展示。
print(tabulate(df_results, headers = 'keys', tablefmt = 'pretty'))

+---+-----------+--------------+--------------+
|   |  Sudoku   | CSP Time (s) | SAT Time (s) |
+---+-----------+--------------+--------------+
| 0 | Sudoku 1  |    0.014     |    0.036     |
| 1 | Sudoku 2  |    0.011     |    0.022     |
| 2 | Sudoku 3  |    0.258     |    0.089     |
| 3 | Sudoku 4  |    0.095     |    0.439     |
| 4 | Sudoku 5  |    1.193     |     0.57     |
| 5 | Sudoku 6  |    0.547     |    0.421     |
| 6 | Sudoku 7  |    0.943     |     0.43     |
| 7 | Sudoku 8  |    4.498     |    0.506     |
| 8 | Sudoku 9  |    0.006     |    0.011     |
| 9 | Sudoku 10 |    1.605     |    0.686     |
+---+-----------+--------------+--------------+
